<a href="https://colab.research.google.com/github/dylanwalker/MGSC496/blob/main/MGSC496_R03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Your Info

your_name = '' #@param {type:"string"}
your_email = '' #@param {type:"string"}
today_date = '' #@param {type:"date"}


# How to "read" this notebook

As you go through this notebook (or any notebook for this class), you will encounter new concepts and python code that implements them -- just like you would see in a textbook. Of course, in a textbook, it's easy to read code and an explanation of what it does and think that you understand it.
<br />
<br />

### Learn by doing
But this notebook is different from a textbook because it allows you to not just read the code, but play with it. **You can and should try out changing the code that you see**. In fact, in many places throughout this reading notebook, you will be asked to write your own code to experiment with a concept that was just covered. This is a form of "active reading" and the idea behind it is that we really learn by **doing**. 
<br />
<br />

### Change everything
But don't feel limited to only change code when I prompt you. This notebook is your learning environment and your playground. I encourage you to try changing and running all the code throughout the notebook and even to **add your own notes and new code blocks**. Adding comments to code to explain what you are testing, experimenting with or trying to do is really helpful to understand what you were thinking when you revisit it later. 
<br />
<br />
### Make this notebook your own
Make this notebook your own. Write your questions and thoughts. At the end of every reading notebook, I will ask the same set of questions to try to elicit your questions, reaction and feedback. When we review the reading notebook in class, I encourage you to   



# Code Preface

In [ ]:
%%capture
!pip install parsel
!pip install scrapy
!pip install colab-xterm
import parsel
from parsel import Selector
%load_ext colabxterm

# Intro

A webscraper's fundamental task is to pull data (text, images, etc.) from various parts of webpages. To do this, we will need to write code that tells our scraper how to find and extract the data. As we will see, this can include a complex chain of logic, that might sound like this: 

>First go to *\<these sites\>*, then in each site, follow the link in *\<this particular place\>* to get to a new page, then grab all of the text that appears in *\<this type of structure\>* and all of the images that appear in *\<this other type of structure\>*.

That probably sound a little vague right now, but will become clear when we build our own scraper. 

Here is how we will approach it:

* First, we will learn about navigating and locating different parts of webpages, by inspecting their html and using the xpath language to "point" our code to different regions of the page.
* Next, we will learn about the Scrapy framework and learn how to write simple code to crawl and extract data from webpages
* Finally, we will learn how to use Scrapy tools and how to deploy our own scrapers, even right here in a colab notebook.


# Navigating HTML docs with XPath


**<font size=4> Inspecting HTML docs with your browser </font>**

If we want to scrape the web and extract useful data from webpages, we have to peer under the hood and see the html that the browser sees and uses to render the page. We will want to leverage patterns in the structure of the html in order to identify and locate the data (text, images) that we want to extract. The first thing that we are going to learn is how to inspect the html of web pages in our browser.

Open up a new tab in your web browser and go to http://quotes.toscrape.com, then open the inspector. You can do this is Chrome, Safari or Firefox by right-clicking somewhere on the page and selecting "Inspect" or "Inspect element". In chrome, it will look something like this:

<img alt="R03_inspecting_webpage.png" src="https://drive.google.com/uc?id=1jwd7vRskmHTt8horeOKeDKjh8pUVbbkF" width="100%">

In the inspection window, you can browse the element tree and click the down arrow to peer further down the branch in an element. Hovering over an element will show which the region of the webpage it corresponds to in the left pane. If you right click on an element in the inspector, you can even copy a snippet of "code" that specifies where that element is in the hierarchy. There are several ways to do this. To copy the xpath of the element, you would right click and select Copy->Copy XPath, like this:

<img alt="R03_copy_xpath.png" src="https://drive.google.com/uc?id=1ovWb_cvNQueCj3BRdOeAyV3yN84JGqVX" width="45%">

This is the xpath that is copied when I do this for the above:
>**/<font>html/body/div/div[1]/div[1]</font>**

This "xpath" is actually written in a (very basic) language that we can use to tell our scrapy program which part of a page to focus on. Let's talk about that now.

<br />








**<font size=4> Introducing XPath</font>**

How can we specify where we want our scraper to look? To do so, we will use selectors to "select" different parts of the page. We have two options for languages to specify our selectors, we can use [css (cascading style sheets](https://en.wikipedia.org/wiki/CSS)) selectors or [xpath](https://en.wikipedia.org/wiki/XPath) selectors. They are just different ways to specify which element/s in a page we want to focus on and extract data from. I'm going to concentrate on xpath here, but css selectors are also an option you can explore and learn about on your own. 

The first thing to note is that an xpath selector may select **one or more** elements on the page (we'll use the terms element and tag interchangeably). This means that when we specify an xpath, we can expect to get back a list of one or more elements of the page (or maybe even zero elements, if nothing matches the path we specified).

<br />

**<font size=3> XPath: Getting children with `/`</font>**

Following the [example in wikipedia](https://en.wikipedia.org/wiki/XPath#:~:text=if%20read%20carefully.-,Abbreviated%20syntax,-%5Bedit%5D), if our page looked like this:

>\<a\><br/>
&emsp;&emsp;\<b\><br/>
&emsp;&emsp;&emsp;&emsp;\<c /\><br/>
&emsp;&emsp;\</b\><br/>
\</a\><br/>

The xpath for the element C would be:
> */a/b/c*

where the `/` denotes "child of". The xpath says: we are looking for an element `c` that is a child of element `b` that is a child of the root node element `a`.

<font size=2> Note: I'm using \<a\>, \<b\>, and \<c\>, but in a real example, these would be html tags like \<div\>, \<span\>, \<img\>, \<a\>, etc.
</font>

Let's try this out using an html parsing library called `parsel`. It isn't installed by defauly on colab, so you'll need to run a snippet of code first that will silently install and import the `parsel` library for you. 

In [ ]:
# Run this cell to setup parsel (you may need to rerun it if your instance disconnects)
%%capture
!pip install parsel
import parsel
from parsel import Selector

***One more thing to note***: Parsel is meant to work with documents that conform to the html standard, so it silently wraps documents with `<html><body></body></html>` tags if they are not already present in the document. I'll show you that it does this in the code below. But for clarity, all future examples will have html and body tags in the document definition. We'll have to include these in our xpath, so our example will differ a tiny bit from the one above.

Ok, now run the code below.

In [ ]:
# Try playing with xpath selectors
doc = """
<a>
  <b>
    <c />
  </b>
</a>
"""
print(doc)
selector = Selector(doc)
print(selector.get()) # this will show that parsel wraps the document with <html><body></body></html> tags ; we'll talk about .get() later
selector.xpath('/html/body/a/b/c') # modify this code to select the a or b element

The output of `selector.xpath(...)` is a `selector list`. In this case, the list has just one element, which is a Selector object. The selector locates the `c` tag on the page. 

<br />

**Why is the result of a xpath selector a list of xpath selector?**

A: It's a list because there may be more than one match to our xpath. Each match is a selector so that we can keep applying xpath rules to it, if we wanted to. In the example above, we could continue doing xpath operations on the selector for `c`. Though in this case, `c` doesn't enclose anything and has no attributes, so there isn't much more we could do. 

<br />

**What's the deal with the `data=...` printed out by the selector?**

You may have noticed that the selector has something called "data". What is this? For tags, the data of a selector is "the tag and everything inside of it". When we want the "data" inside a selector we can use the selector's `.get()` method:

In [ ]:
sel_list = selector.xpath('//a/b/c') # select any elements that match the xpath. The result will be a list of selectors
sel = sel_list[0] # get the first element in the list, this will also be a selector
sel.get() # get the data of the element located
# modify the above code to see what happens when you select the a or b element. What does the "data" look like?

<hr/>
<img src="https://drive.google.com/uc?id=1sk8CSP26YY7sfyzmHGFXncuNRujkvu9v" align="left">

<font size=3 color="darkred"> Test Your Xpath </font>

<font> Using the same document above, write code to select the `a` element and get its data.
</font>    

In [ ]:
# Enter your code here

<hr/>


**<font size=3> XPath: Indexing Elements with `[]`</font>**

Similarly, suppose the page looked liked this:

>\<html>\<body><br/>\<a\><br/>
&emsp;&emsp;\<b\><br/>
&emsp;&emsp;&emsp;&emsp;\<c /\><br/>
&emsp;&emsp;\</b\><br/>
&emsp;&emsp;\<b\><br/>
&emsp;&emsp;&emsp;&emsp;\<d /\><br/>
&emsp;&emsp;\</b\><br/>
\</a\><br/>
\</body>\</html>

The xpath for the `b` elements would be:
>*/html/body/a/b*

And if we wanted specifically the path to just the 2nd `b` element, the xpath would be:
>*/html/body/a/b[2]*

The ***square brackets "index" which elements we want that matches, starting at 1 for the first*** (note: when indexing python lists, we start at 0, but in xpath, the index starts at 1).

<br />

Try it out:

In [ ]:
doc = """
<html><body>
<a>
  <d />
    <b>
      <c />
    </b>
    <b>
      <d />
    </b>
</a>
</body></html>
"""
print(doc)
selector = Selector(doc)
selector.xpath('/html/body/a/b')

In [ ]:
sel_list = selector.xpath('/html/body/a/b') # this will be a "selector list" of two selectors. 
#Try checking the type of sel_list. What is it?

When you check the type of the above, you will notice that **this isn't a regular python list, but instead is a "selector list"**. This is a list-like object that behaves like a regular python list, but has some extra spice added in. Here's one cool trick: If you want the data from every item in a selector list, you can call the selector list's `.getall()` method:

In [ ]:
sel_list.getall() # returns a list of the data. What is the type of this object?

['<b>\n      <c></c>\n    </b>', '<b>\n      <d></d>\n    </b>']


**<font size=3> XPath: Finding descendents with `//`</font>**

In addition to using `/` to specify a child relationship, we can also use `//` to specify any deeper descent.

Suppose the page looked like this:

>\<html\>\<body\><br/>\<d /><br/>
\<a\><br/>
&emsp;&emsp;\<d /\><br/>
&emsp;&emsp;\<b\><br/>
&emsp;&emsp;&emsp;&emsp;\<c /\><br/>
&emsp;&emsp;\</b\><br/>
&emsp;&emsp;\<b\><br/>
&emsp;&emsp;&emsp;&emsp;\<d /\><br/>
&emsp;&emsp;\</b\><br/>
\</a\><br/>
\</body\>\</html\>
<br />

If we want to match the 2nd and 3rd `d` (that are both enclosed by an `a`), but not the first `d`, we can do so with the xpath:
>*/html/body/a//d*

because both `d`'s are a descendent of an `a` (even though they aren't direct children). 

More importantly, `//` is even more useful because it allows us to "start anywhere". If we just wanted to find *any* `d` tag that was descendant of an `a` tag, regardless of whether or not that `a` tag was just under `/html/body`, we could specify it like this:

>*//a/d*

You will see many xpaths like this that ***do not reference the root node of the document***. 


In [ ]:
doc = """
<html><body>
<d />
<a>
  <d />
    <b>
      <c />
    </b>
    <b>
      <d />
    </b>
</a>
</body></html>
"""
print(doc)
selector = Selector(doc)
selector.xpath('//a/d')


<br/>

**<font size=3> XPath: Wildcards</font>**

We've been using explicit tags in all the xpath examples, so far, but it is also possible to use the wildcard `*`. Using the page from the last example, if we wanted to select every tag that was a child of an `a` element (in the above, this would be the second tag `d` and the two `b` tags), we could do so with the xpath:
>*//a/\**



In [ ]:
selector.xpath('//a/*') # Select every tag that is a child of an a 
# Now modify the above to instead select every tag that is a descendant of a

<hr/>
<img src="https://drive.google.com/uc?id=1sk8CSP26YY7sfyzmHGFXncuNRujkvu9v" align="left">

<font size=3 color="darkred"> Test Your Xpath </font>

<font> Using the same document above, write a single line of code that returns a selector list of every element that is a **descendant** of an element `a`
</font>    

In [ ]:
# Enter your code here

<hr/>

**<font size=3> XPath: Selecting tags based on attributes, using `[@attr ...]`</font>**

The most useful thing that xpath will let us do is to select nodes/tags conditionally, and especially based on the value of attributes of tags. Recall that html tags have attributes, like this:

`<a href="http://wikipedia.com" class="reflink">This is a link</a>`

here the two attributes of the **a** tag are `href` and `class`. Class attributes are used widely in modern webpages and are a great "hook" for us to find a reference to the part of the webpage we want. For example, suppose we had a webpage which had a bunch of reference links in it, and we knew that each one had the attribute `class="reflink"`, we could make an xpath selector that would yield a list of all of those links using this xpath:
>*//a[@class="reflink"]*

Note that we can put all kinds of expressions inside of the square brackets. For example, suppost we had a document that had lots of images of different sizes, and we wanted to extract all images smaller than some width (maybe these are thumbnails we want to download), we might do so with an xpath like this:

>*//img[@width<300]*

Of course, you can also combine multiple conditions using `and` and `or` in expressions inside the brackets.

<br/>


**<font size=3> XPath: Getting attributes and text within tags</font>**

Sometimes you may want to extract the actual attribute of a tag using xpath. Suppose you had some html with a bunch of span tags, like this:

```
<div class="toyblock>
  <span class="imgwrapper"><img src="voltron.png" /></span>
  <span class="toyname">Voltron</span>
  <span class="price">$14.99</span>
</div>
<div class="toyblock>
  <span class="imgwrapper"><img src="he-man.png" /></span>
  <span class="toyname">He-Man</span>
  <span class="price">$5.99</span>
</div>
```
and you wanted to get the `class` attribute of the span tags under a `toyblock` div, you could do it like this:
>//div[@class="toyblock"]/span/@class

This would give you all of the element class attributes that were children of the `toyblock` div tag.

Suppose you wanted to get the text inside the span tags for the toynames? You can do that with the by adding `text()` to the end of the xpath, like this:
>//span[@class="toyname"]/text()

You can even conditionally match elements based on their inner text using `text()` inside square brackets, though this is less common because inner text in webpages may not match a common pattern.

<br/>


In [ ]:
doc = """
<html><body>
<div class="toyblock>
  <span class="imgwrapper"><img src="voltron.png" /></span>
  <span class="toyname">Voltron</span>
  <span class="price">$14.99</span>
</div>
<div class="toyblock>
  <span class="imgwrapper"><img src="he-man.png" /></span>
  <span class="toyname">He-Man</span>
  <span class="price">$5.99</span>
</div>
</body></html>
"""
print(doc)
selector = Selector(doc)
selector.xpath('//span[@class="toyname"]/text()').getall() # get a list of the inner text of span tags with class="toyname"

<hr/>
<img src="https://drive.google.com/uc?id=1sk8CSP26YY7sfyzmHGFXncuNRujkvu9v" align="left">

<font size=3 color="darkred"> Test Your Xpath</font>

<font> Using the same document as in the above code, write a single line of code that generates a list of the image .png filenames for the toys on the page.   
</font>    

In [ ]:
# Enter your code here


<hr/>

**<font size=3> XPath in the real world</font>**

Ok, we have seen enough "artificial" examples. Let's move to some real world ones. We need a way to grab pages from the web. We will do this with the `requests` module. Let's use that to grab the content of the page we want to scrape, http://quotes.toscrape.com. Don't forget we can inspect this page in our browser. You should do so now to compare the results of the code below with what you see in the inspector and on the page.


In [ ]:
# Grab the webpage and make a selector object from its html 
import requests
page = requests.get('http://quotes.toscrape.com') # fetch the webpage "http://quotes.toscrape.com"
doc = page.text # store the raw html of the page
selector = Selector(doc) # make an xpath selector object from the raw html

New we can do any selector we want. First, let's try to get a sense of how the designer of this webpage used classes. We'll start with the `<div>` tag, and see what classes they gave it:

In [ ]:
# Check what classes div tags have on the page we want to scrape
div_classes = selector.xpath('//div/@class').getall() # grab a list of all of the div classes that occur on the page
div_classes = list(set(div_classes)) # this trick only keeps the unique elements of the list, by converting it to a set and then back to a list
print(div_classes)

['col-md-4 tags-box', 'quote', 'tags', 'col-md-8', 'row', 'row header-box', 'col-md-4', 'container']


The first thing we notice is that some `<div>`'s have multiple classes. For example, `'col-md-4 tags-box'` is actually two classes. If we need to find elements that have multiple classes we can't just use an xpath with `[@class='...']`. The `parsel` documentation [has some suggestions about dealing with multiple classes](https://parsel.readthedocs.io/en/latest/usage.html#when-querying-by-class-consider-using-css), but for now we'll ignore these.

If we want to scrape the quotes, the most relevant class seems to be `quote`, so let's check it out:

In [ ]:
# What does the html inside one of these "quote" div tags look like?
selector.xpath('//div[@class="quote"]').get() # .get() applied to the selector list will give us the data of the first element of the list

From the above, we can see that the actual quote seems to be enclosed in a `<span class="text">` tag, so let's get the text enclosed in all of those: 

In [ ]:
quotes = selector.xpath('//div[@class="quote"]/span[@class="text"]/text()').getall()
print(quotes)
len(quotes)

Perfect! Those are all the quotes on the page, now in a nice list. We would still like to get the authors and the tags for each quote.

<hr/>
<img src="https://drive.google.com/uc?id=1sk8CSP26YY7sfyzmHGFXncuNRujkvu9v" align="left">

<font size=3 color="darkred">Use xpath to get a list of the authors of all the quotes</font>

<font> Using your browser inspector and python code find a way to:

    * create a list of all of the authors of the quotes.
    * create a list of all of the links to about for each author
</font>    

In [ ]:
# Enter your code here


<hr/>

Now that you've gotten a taste for using xpath, we're ready to start talking about the Scrapy webscraping framework.

# Scrapy

**<font size=4>What is Scrapy?</font>**

<img alt="R03_scrapy_logo.png" src="https://drive.google.com/uc?id=1OcMETu2Q6TS9717-gG5EENhePNc7fHGx" width="30%">


[Scrapy](https://scrapy.org/) is a framework for writing your own webscrapers. It is incredibly fast, incredibly powerful, and easily extended to suit many projects. For some small scraping tasks, it may even seem like overkill. However, once you learn how easy it is to make a scrapy crawler project with your own spider and item data definitions, you will be tempted to use it even for small tasks.  

**<font size=4>Scrapy Components</font>**

We will learn about several components of Scrapy, including:
* *Spiders*: Classes that define how a site will be scraped (how to "crawl" or follow links, and how to extract data from pages)
* *Selectors*: These are the same xpath selectors you just learned about.
* *Items*: A python class that reflects the structure of the data objects you want to extract (e.g., a quote has the body of the quote, an author field, tags, etc.)
* *Scrapy command line*: A command line tool that lets you create and launch scrapy projects and specify crawling options, such as where to store the scraped data.
* *Scrapy shell*: An interactive python shell that can be launched to explore webpages and how your spider code interacts with them. It can also be launched when your scraper encounters an exception, to let you explore what might have gone wrong.


<br />

**<font size=4>Scrapy Architecture</font>**

First, I'll give you the description of how Scrapy works directly from the mouths of the developers, then I'll give you my own simplified description. Don't worry if you don't understand the developer's description, ***just skim it to get an idea.*** The [developers say](https://docs.scrapy.org/en/latest/topics/architecture.html):

><img alt="R03_scrapy_architecture.png" src="https://drive.google.com/uc?id=1p0hLGUdUjfkJiVcEAmpfZ6GgebP9FXTK" width="100%">
The data flow in Scrapy is controlled by the execution engine, and goes like this:
>
>1. The Engine gets the initial Requests to crawl from the Spider.
>
>2. The Engine schedules the Requests in the Scheduler and asks for the next Requests to crawl.
>
>3. The Scheduler returns the next Requests to the Engine.
>
>4. The Engine sends the Requests to the Downloader, passing through the Downloader Middlewares (see process_request()).
>
>5. Once the page finishes downloading the Downloader generates a Response (with that page) and sends it to the Engine, passing through the Downloader Middlewares (see process_response()).
>
>6. The Engine receives the Response from the Downloader and sends it to the Spider for processing, passing through the Spider Middleware (see process_spider_input()).
>
>7. The Spider processes the Response and returns scraped items and new Requests (to follow) to the Engine, passing through the Spider Middleware (see process_spider_output()).
>
>8. The Engine sends processed items to Item Pipelines, then send processed Requests to the Scheduler and asks for possible next Requests to crawl.
>
>9. The process repeats (from step 3) until there are no more requests from the Scheduler.

<br />

**<font size=4>Simplified Scrapy Architecture</font>**

Since we will be only writing code for the Scrapy spider, we can only focus on this and ignore how the rest of the Scrapy engine works. Here's my own take on how a scrapy spider works:


<img alt="R03_scrapy_architecture_dylan.png" src="https://drive.google.com/uc?id=1bLeUINw85Mi_6AkJB3m-pGFmGEilx67o" width="100%">
Requests are shown as orange lines. Responses are shown as green lines. Data items are shown as dashed blue lines.


1. The spider begins by taking the `start_urls` one by one, making them into request objects and feeding them to the back of the line of the **request queue**

2. A `request` object is taken from the front of the line. Each request specifies not only the url but also which `callback` method of the spider will be used to handle the `response`. The request is then sent out through the internet to the url's webserver.

3. The url's webserver generates a `response` to the request. This response has a reference to the request that generated it. So it knows which callback method was specified. Scrapy calls the specified callback method and passes it the `response` as an argument.

4. Parse methods handle responses. They may sort through the html of the response (e.g., with `.xpath()`) and can yield **data items** (things we want to save to files) and even **generate further requests** (e.g., if the page has a "next" link, generate a new request with the next link as its url). Any new requests that are generated will have a callback method specified to determine who will handle their response and will be fed onto the back of the line of the request queue.


**<font size=4>Writing Simple Scrapy Spiders</font>**

What does this mean for us practically? It means that to write a webscraper with scrapy, all we have to do is to fill in the blanks of some Spider class code by writing some parse methods. Here is what the code from a "blank" Scrapy Spider looks like:

```python
# MySpider.py -- a blank Scrapy Spider
import scrapy

class MySpider(scrapy.Spider):
    name = 'myspider'
    allowed_domains = ['some.domain.com']
    start_urls = ['http://some.domain.com/somepage']

    def parse(self, response):
      # YOUR CODE HERE TO: parse the response, yielding data items and/or other requests
      pass
```

In other words, we just have to write code for what the `parse()` method does. Sometimes, we might have only a single parse method. Sometimes, we will want to write more (`parse2()`, etc.) and assign them as callbacks to different types of requests. 

You might be wondering how we can get data out of this. The simplest way is to just pull the data out by using xpath operations, assign this to python variables, and then make a dictionary where the keys are the names and the values are the python variables we want to save. Here's ***an example for a hypothetical website that we want to pull toy names and prices from***.   

```python
# ToySpider.py -- a hypothetical spider for scraping names and prices from a toy site
class ToySpider(scrapy.Spider):
    name = 'toyspider'
    allowed_domains = ['cool.toys.com']
    start_urls = ['http://cool.toys.com/new']
    
    def parse(self,response):
      sel_list = response.xpath('/xpath/to/toyitem')
      for toy_sel in sel_list:
        toyname = toy_sel.xpath('continuedpath/to/toyname/text()').get()
        toyprice = toy_sel.xpath('continuedpath/to/toyprice/text()').get()
        yield {'toyname': toyname, 'toyprice': toyprice}
```

Let's walk through what it does. First it uses xpath to locate a  'toy item' block. Then it loops over every such toy item and uses relative xpaths to find the name and price of the toy in the block. Lastly, it creates a dictionary with the toyname and toyprice and `yield`'s it. Remember, a yield is like a `return` that doesn't end the function.

Of course, this spider would only be able to handle a single page (the single `start_url` we gave it). We could have added more urls to that `start_url` list, but what if we didn't know what they were? What if we just knew that there was a "next" button somewhere on the page and we wanted our spider to "click it" (i.e., follow its link and then process the page you landed on). We could add some code to do that, like this:

```python
# ToySpider.py -- a hypothetical spider for scraping names and prices from a toy site
class ToySpider(scrapy.Spider):
    name = 'toyspider'
    allowed_domains = ['cool.toys.com']
    start_urls = ['http://cool.toys.com/new']
    
    def parse(self,response):
      sel_list = response.xpath('/xpath/to/toyitem')
      for toy_sel in sel_list:
        toyname = toy_sel.xpath('continuedpath/to/toyname/text()').get()
        toyprice = toy_sel.xpath('continuedpath/to/toyprice/text()').get()
        yield {'toyname': toyname, 'toyprice': toyprice}
      # Here is some code to find the next link and yield a request for it:
      next_relative_url = response.xpath('/xpath/to/next/link/@href').get() 
      next_url = response.urljoin(next_relative_url) 
      yield scrapy.Request(url=next_url,callback=self.parse)
```

In the above, to follow the next link, we had to extract the url of the "next" link from the response html, and then conver it from a relative url to an absolute one (e.g., `/page/2` is relative in that you assume it has the same domain as the page the link is on, whereas an absolute url would be `quotes.toscrape.com/page/2` ) and then yield a new request. In fact, since following a link from the present page is such a common practice, the scrapy response object has a method which combines these last two lines of code into one:
```python
      yield response.follow(url=next_url, callback=self.parse)
```

In both of the hypothetical examples shown above, we used only a single method `parse` to handle responses, but we could have defined any number of methods with whatever method names we wanted. 

Now that we have the basic idea of how to write a scraper, let's looks at how we can use and deploy scrapy.

**<font size=4>Using Scrapy in Google Colab</font>**

Scrapy projects are made up of several (object oriented) pieces of python code (`.py` files) that work together. As is common with OOPs (objected oriented projects), class definitions are usually stored in their own `.py` python files. Typically, such projects are wrapped up in a module structure, so that they can be distributed this way and imported by others. There is usually an entrypoint for such projects -- some particular `.py` file that you run to start everything running. Scrapy has its own command line tool for creating and launching projects. For example, to start a scrapy crawler for which you have already written the code, you would call `scrapy crawl projectname` from within the project directory of an operating system shell/command prompt (e.g., osx terminal, windows command line, or even, as we will see, a colab xterm terminal or `!command` in a colab cell)

Using Scrapy from Colab is a bit funny, because Scrapy is not designed to be run as a python notebook. But in practice, its very easy and convenient to get it to do so.

<br />

**<font size=3>Install Scrapy</font>**

First we need to make the google machine instance that we are using with our notebook install scrapy, because it is not one of the libraries that is pre-installed.

The following code will do this. You will notice that several other libraries  will be installed by this command. These are Scrapy's *dependencies*

<font size="2">*Pro tip: sometimes a code cell in colab will produce a lot of output that you would rather not see. If you want to silence the output of a code cell, you can just make the first line</font>* `%%capture`


In [ ]:
!pip install scrapy


Also, we will want to use the scrapy command line tools, so we will need some access to a system terminal (i.e., "shell" or "command line" access).  Unfortunately, the free version of coalb doesn't come with a terminal. If you pay for **colab pro**, there is a convenient terminal button on the side menu (the same sidebar menu that has the file explorer). As an alternative, we will install the `colab-xterm` library and load it as an extension to colab:

In [ ]:
%%capture
# Install colab-xterm
!pip install colab-xterm
%load_ext colabxterm


Great, we're ready to go.


**<font size=3>Scrapy Command Line: Scrapy Shell</font>**

Scrapy has a command line tool that we can use to do a number of things to get our scrapy project up and running. Check out the [scrapy command line documentation](https://docs.scrapy.org/en/latest/topics/commands.html#command-line-tool). 


The first Scrapy command we'll talk about is running a scrapy shell. Scrapy shell is an interactive python environment with a default scrapy crawler invoked, giving access to many of the objects that are used by the scrapy engine. It's a good place to experiment with xpath and develop the spider code that will parse your webpage. [Remember earlier](#scrollTo=vUObxVFSv4jz) when we used the `requests` and `parsel` libraries to extract data from `quotes.toscrape.com`? Scrapy shell provides a similar environment with some extra access to the other objects that Scrapy uses. Scrapy shell can also be called from your scraper when things go wrong to help you "debug" it. 

<br />


Let's run Scrapy shell now. If we were doing this on a local installation of python, we would just open up a terminal or "command prompt" (e.g., terminal in osx; cmd in windows) and type the command: 

`scrapy shell websiteaddress`

But since we are in colab, we will use a `colab-xterm` to do this. Run the below code cell to launch the terminal and inside that terminal type:


In [ ]:
%xterm

In the shell above, we are going to explore the html `response` received by vising the site [quotes.toscrape.com](quotes.toscrape.com). To do this **in the terminal window that opens above**, type the following command:
```
scrapy shell "quotes.toscrape.com"
```
Your output should look something like this:

<img alt="R03_scrapy_shell_in_xterm.png" src="https://drive.google.com/uc?id=1D9DmAlVJR6v9NHVlj4D-E6yQP2Fbt5fL" width="100%">

<font color="darkred" > **If you'd prefer to run the colab xterminal in a separate window so that you can keep it open and tab between it and this notebook, I have provided a notebook `MGSC496-ScrapyXterm.ipynb` with just scrapy and a terminal.** Note that colab will generally let you edit any number of notebooks at the same time, but will only yet you have one running (unless you pay for colab pro).</font>

Scrapy shell tells you about the various objects that are defined in this interactive environment.These are: the `scrapy` module, and some objects such as the `crawler`, `spider`, and `settings` objects which we'll talk about later, as well as the `request` and `response` objects, which relate directly to the webpage that we told scrapy shell to fetch for us. We sent a web **request** to `quotes.toscrape.com` and we got back a **response**.  We can do this for any url from within scrapy shell by using the `fetch('some.website.com')`. 

If we were running Scrapy on a local machine instead of in a cloud notebook, we would be able to view the page we fetched in our system's default browser using `view(response)`. However, this won't work in colab. Instead, we can simply open the page we fetched in a browser on our local machine if we would like to look at the page and inspect the html. We will need to do this to design our scraper.

Let's explore the `request` and `response` objects in scrapy shell in our terminal window.

<hr/>
<img src="https://drive.google.com/uc?id=1sk8CSP26YY7sfyzmHGFXncuNRujkvu9v" align="left">

<font size=3 color="darkred"> Explore the `request` and `response` objects in Scrapy Shell </font>

<font> What kinds of things are in the `request` and `response` objects? Use methods like `type`, `dir` `id` and `print` functions to explore these objects 
</font>    

Double click this text box to edit it to add your answers to the questions below

* Explain what each of these is, in your own words:

 * `request.url`: 
 * `request.headers`:
 * `request.method`:
 * `response.url`:
 * `response.ip_address`:
 * `response.headers`:
 * `response.status`:
 * `response.body`:
 * `response.request`:

* Why does `response` have a `request` object inside of it? 

* Is it the same as the request object or is it different? (hint: [use python's `id()` function](https://docs.python.org/3/library/functions.html#id) to answer this question).

* Use `fetch()` to fetch a different page from within scrapy shell

<hr/>

**<font size=3>A Brief Aside about HTTP</font>**

*If we're going to scrape webpages, we need to know a little about the http protocol. I will keep this super brief at the risk of falling down a rabbit hole.*

To get a webpage, we form a web request which includes the url of the page we want and the http method we are going to use to request it (either a `GET` or `POST`). Our request may carry some information to the webserver (that includes what kinds of things our client machine can do -- what we "Accept" -- what kind of language pages we accept -- "Accept-Language" and even what kind program is asking (e.g., a browser) -- this is contained in the "user-agent" header field. All of this extra information lives in `http headers`. **Have a look at `request.headers` in the shell now.**

When the webserver recevies our http request, it sends back an http response message. This message also has headers. Have a look at them now in the shell. In addition, it contains a status field, with a code that indicates if there was a problem with the request/response. [Here is a list of different http codes](https://en.wikipedia.org/wiki/List_of_HTTP_status_codes). And, of course, the response contains the body of the page that you asked for itself (in html), under `response.body`.**Have a look at `response.body` now in in the shell.** You will see it is a lot of html that is sorta human readable but not easily. Fortunately, you have the browser inspector and xpath to help you play with it. 

<br />


**<font size=3>Scrapy Command Line: Making a new project</font>**

Scrapy projects are made up of several files. But we only need to customize a little bit of code to make our scraper. We can generate a blank new project using the scrapy command `scrapy startproject projectname` like this:


In [ ]:
!scrapy startproject quotescraper

We can immediately see all the files this created by looking at the colab file browser:


<img alt="R03_scrapy_project_files.png" src="https://drive.google.com/uc?id=1g71vYBhY_R8Op1Zas1holjNiS9kHtik6">


You can see that a Scrapy project has:
- `/spiders/`: A place put your spider code, which governs the behavior of one or more "spiders" that crawl the web
- `settings.py`: A place to configure scrapy project settings
- `items.py`: A place to define the data structure of stuff that we will extract with our scraper (we can define this and yield our own special item objects, instead of just yielding a dictionary)
- We won't discuss the other two (`pipelines.py` and `middlewares.py`)

For many simple scrapers, all we need to do is run this startproject command and then write code for a single spider and save it as a `.py` file in this `/spiders/` folder. We can write all the code from a cell in colab to a file using the `%%writefile` command as the first line of the cell.

Run the code below:



In [ ]:
%%writefile /content/test.py
print("This is a test.")

Writing /content/test.py


Now use the file browser in colab to verify that the file `/content/test.py` was created and open it to see its contents. Once you have done so, you can safely delete this file if you want. Notice that the default home directory for colab is `/content`.

One annoying thing about colab is that, when the first line of the code cell is `%%writefile`, you will lose all the nice python context (code will no longer be colored to highlight syntax). Fortunately, we can comment out the first line like this `#%%writefile` to get our colors back. Once we're done working with the code in that cell, we can remove that  the `#` and run the cell to write the code to the file.

We will use this trick to write our `quotespider.py` file.

**<font size=3>Scrapy Command Line: Generating stub code for a spider</font>**

You don't have to write the code for your spider completely from scratch, or even copy and paste starter code from somewhere else, because scrapy command line also has a tool that will generate some boilerplate code for your spider. Just enter the directory of your project and run:
```
scrapy genspider spidername somedomain.com
```

Let's do this now to see the boilerplate code that it generates. Don't forget that when you want to execute a shell command in a specific location in colab, you should use `&&` chaining to change to the desired directory and then execute the command. Try running the following:




In [ ]:
!cd /content/quotescraper && scrapy genspider quotespider quotes.toscrape.com

Created spider 'quotespider' using template 'basic' in module:
  quotescraper.spiders.quotespider


Now open the file `/content/quotescraper/quotescraper/spiders/quotespider.py` by finding it in the colab file explorer and double clicking it.

This is helpful if its been a while since you've used scrapy and you want a quick start. When we work on scrapy projects in colab, we will use the `%%writefile` trick discussed above to write our spider code to a `.py` file in the correct location, so we won't use genspider. If you want to use it, you can generate the spider code and then copy and paste it to a cell.

**<font size=3>Scrapy Command Line: Running a crawler project</font>**

Once we've written our code for the spider, we can run our project from the command line by entering the top level directory of the project and running the command `scrapy crawl projectname`. Typically, we will want to save the output of a scraper to a file. We can specify this with the `-o` or `-O` option:
```
scrapy crawl projectname -o outputfilename.json
```
The above will output data to the json file (default format). Using `-o` will append data to the end of the file if it exists already, whereas `-O` will overwrite the file. To see other options, check out  the [documentation for the scrapy crawl command](https://docs.scrapy.org/en/latest/topics/commands.html#crawl).


Remember that scrapy commands should always be run in your project folder, so we would use:
```
!cd /content/quotescraper && scrapy crawl quotescraper -o output.json
```

<hr/>
<img src="https://drive.google.com/uc?id=1sk8CSP26YY7sfyzmHGFXncuNRujkvu9v" align="left">

<font size=3 color="darkred"> Your First Scrapy Scraper</font>

<font> You should now know enough to write your own very simple scraper. Treat the boxes below as if you are writing in a completely blank colab notebook. What do you need to do to write and run your own scraper? We will be focusing on `quotes.toscrape.com`. As you can see from browsing the site, each quote has content, the author, and tags; each author also has an about page. For now, your job is write a scraper with a spider that just scrapes the content (the body of the quote itself) and the author. Make sure your spider finds the next page link and follows it.  
</font>    


1. Make sure scrapy is installed in colab:

In [ ]:
# ENTER CODE HERE

2. Create a new scrapy project:

In [ ]:
# ENTER CODE HERE

3. Write your spider code to a file in the appropriate directory:

In [ ]:
# ENTER CODE HERE

4. Tell scrapy to start crawling:

In [ ]:
# ENTER CODE HERE

You can test out your scraper by making a new colab notebook, copy/pasting your code in the cells above and running it.

<hr/>

# Feedback
What did you think about this notebook? What questions do you have? Were any parts confusing? Write your thoughts in the text box below.

<font size =2> note: You can double click this text box in colab to edit it.</font>

PUT YOUR THOUGHTS HERE

# Submit
Don't forget to submit your notebook before class! Make sure you have saved your work (**Colab Menu: File-> Save**) and then download a pure python copy (**Colab Menu: File-> Download -> Download .py**)

